In [ ]:
# Import libraries
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
import tqdm

In [ ]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device='cpu'
print(device)

In [ ]:
main_folder = "/kaggle/input/iith-dl-contest-2024/train/train"

In [ ]:
# Define a transform to read the data in as a tensor
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# Load the training data
train_data = datasets.ImageFolder(main_folder, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
print("Data loaded")
print("Number of training samples: ", len(train_data))
# print shape of training data
print("Shape of training data: ", train_data[0][0].shape)
print(train_data.classes)
print("Number of classes: ", len(train_data.classes))

In [ ]:
# print a sample image
import matplotlib.pyplot as plt
image, label = train_data[0]
plt.imshow(image.permute(1, 2, 0))
plt.show()

In [ ]:
test_dir="/kaggle/input/iith-dl-contest-2024/test"

In [ ]:
test_data = datasets.ImageFolder(test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)        
# print(preds)

In [ ]:
# test predictor
def test_set_predict(model, test):
#     model.eval()
    preds = []
    for data, target in test:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        _, pred = torch.max(output, 1)
        preds += pred.tolist()
    return preds

In [ ]:
# function to print the CSV for each epoch 
def printer(model,epoch_no):
    preds=test_set_predict(model,test_loader)
    class_names = train_data.classes
    # generate file names are 1.JPEG, 2.JPEG, etc using for loop
    file_names = [str(i)+".JPEG" for i in range(len(preds))]

    # sort the file names
    file_names.sort()
    with open("/kaggle/working/submission_resnet_"+str(epoch_no+1)+".csv", "w") as f:
        f.write("ID,Category\n")
        for i in range(len(preds)):
            f.write(file_names[i]+","+class_names[preds[i]]+"\n")

In [ ]:
# Do using ResNet18
from torchvision import models

model = models.resnet34(weights=None).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
for epoch in range(n_epochs):
    train_loss = 0.0
    correct = 0
    total = 0
    model.train()
    for data, target in tqdm.tqdm(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    acc = 100 * correct / total
#     torch.save(model.state_dict(), "./26apr_conv_next_tiny/model_"+str(epoch+1)+".pt")
    model.eval()
    printer(model,epoch)
    print('Epoch: {} \tTraining Loss: {:.6f} \tAccuracy: {:.2f}'.format(epoch + 1, train_loss, acc))

In [ ]:
# epoches=9, kaggle=0.51680 
# run this model again upto 9_epoches
# you can find submissions in submit_csvs
# see submissions on date 24apr